# **Import Module**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-w1wi_c_o
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-w1wi_c_o
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/kpmg/concat.csv')

In [ ]:
data

,contents,category
0,이를 유해화학물질 취급시설을 지속적으로 개선하여 화관법 취급시설 최초 검사 결과 발...,e
1,관리 변화와 평가하기 공개합니다 따라 위험과 사용된 기후 기회를 지표를 b. Sco...,e
2,"[투입정보 및 가정, 분석방법] 국내외사업장의 18년 21년 배출량, 디스플레이 업...",e
3,향후 목표한 바를 향해 공백기술 개발 및 수소 활용처 확대를 위한 사업을 지속적으로...,e
4,"2021년 기준 총 3,317명의 인원이 이수하여 및 위해성을 관리하고 있습니다. ...",e
...,...,...
3299,이와 같이 어떤 물체가 양전기와 음전기만을 띠는 대전체로부터 외부에 나타나는 전기적...,중립
3300,이 중 외부 시나리오에 따른 재무적 영향 측정이 필요한 요인은 8개인 것으로 확인되...,중립
3301,(87) 64 (94) 48 (92) 4 (100) 5 (71) 13 (57) 23...,중립
3302,전자결재 시스템과 연동된 전자계약 시스템을 운영하여 사전 품의를 거쳐 정해진 표준계...,중립


In [ ]:
data.loc[(data['category'] == "중립"), 'category'] = 0
data.loc[(data['category'] == "e"), 'category'] = 1
data.loc[(data['category'] == "s"), 'category'] = 2
data.loc[(data['category'] == "g"), 'category'] = 3

data_list = []
for q, label in zip(data['contents'], data['category'])  :
    data1 = []
    data1.append(q)
    data1.append(str(label))

    data_list.append(data1)

In [ ]:
print(data_list[0])
print(data_list[100])
print(data_list[250])
print(data_list[1000])
print(data_list[2500])
print(data_list[3300])

['이를 유해화학물질 취급시설을 지속적으로 개선하여 화관법 취급시설 최초 검사 결과 발생을 사전 예방하기 위해 다양한 노력을 기울이고 있습니다. 통해, 화학 업계의 새로운 패러다임 변화에 적극 대응함으로써 신뢰할 수 있는 ‘적합’ 통보받았으며, 화학물질 누출 등의 사고를 미연에 방지하고자 노력하였습니다.', '1']
['현대자동차는 나아가 해당 에코닐을 당사 차량 소재로 적용하는 방안까지 검토하고 있습니다. 업사이클링 프로젝트 ‘Re:Style’ 해양 생태계 복원 및 업사이클링 프로젝트 © Cor Kuyvenhoven 27 1.', '1']
['기후변화와 관련된 위험과 기회가 조직의 사업, 전략, 재무에 미치는 영향 c. 기후변화 시나리오를 고려한 조직 전략의 회복탄력성 C3.1 기후 변화로 발생하는 위험과 기회를 검토하고 있습니다. 기후변화 리스크를 중대성에 따라 위험단계별로 식별하고 있으며 단기, 중기, 장기로 단계별로 구분하여 관리하고 있습니다.', '1']
['또한 전문기관 컨설팅을 통해 사용계획서 등을 통해 작업 전 사전 승인을 득한 후, 안전관리자가 사업장 안전관리 상태 조사 및 기술 지도를 진행해 안전보건 개선 안전교육을 실시한 뒤에 작업이 진행될 수 있도록 하고 있습니다. 사항을 도출하고 이행하며 재해 발생을 예방하고 있습니다.', '2']
['품질경영회의 # 단순/중대 하자 판단 : 품질조직 팀장 회의 안건 상정여부 결정 지침개정 필요 시 회의 의사결정 필요 Data 분석/검토 즉시 조치 가능 Zero Defect 회의 상품/설계 표준 현장 적용 품질 이슈 발생 ·공사 중 품질이슈(주택 사업부 취합) ·준공 현장 하자(e편한세상품질팀, 아크로품질팀) 단순 오시공 주택사업부 중대하자 및 품질이슈 품질이슈 대책회의 해당 현장 전수조사 및 여건보고 하자인식 및 실적 모니터링 품질경영회의 지침 기준 수립/상품 적용 시공성 + 하자개선 여부 확인 # 본부장 참석/결정안건 보고 전사적 품질경영(TQM) 고객 Action Plan 고객의 소리에 기반한 고객

In [ ]:
from sklearn.model_selection import train_test_split                                                        
dataset_train, dataset_test = train_test_split(data, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

2478
826


In [ ]:
dataset_train.groupby(['category']).count()

,contents
category,
0,495
1,734
2,773
3,476


In [ ]:
dataset_test.groupby(['category']).count()

,contents
category,
0,174
1,266
2,227
3,159


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([dataset.iloc[i][sent_idx]]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i][label_idx]) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# **KOBERT 학습시키기**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


def accuracy_per_class2(X,Y):
  X = np.array([int(i) for i in X])
  Y = np.array([int(i) for i in Y])
  labeldict = {0:'중립', 1:'e', 2:'s', 3:'g'}
  #print(X)
  #print('----------------------------------------------------------------')
  #print(Y)
  for i in np.unique(Y):
    y_preds = X[Y==i]
    y_true = Y[Y==i]
    print(f'Label: {labeldict[i]}')
    print(f'Accuracy: {len(y_preds[y_preds==i])}/{len(y_true)}\n')

Train

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100



In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    #acclist = np.array([0,0,0,0])
    outlist = []
    labellist = []
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        max_vals, max_indices = torch.max(out, 1)
        outlist.extend(max_indices)
        #print(label)
        #acclist += np.array(accuracy_per_class(out, label))
        #outlist.extend(out)
        labellist.extend(label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #print(outlist)
    #print('----------------------------------------------------------------------------------------------')
    #print(labellist)
    accuracy_per_class2(outlist, labellist)
    with torch.no_grad(): 
        model.eval()
        outlist2 = []
        labellist2 = []
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            max_vals, max_indices = torch.max(out, 1)
            test_acc += calc_accuracy(out, label)
            outlist2.extend(max_indices)
            labellist2.extend(label)
        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
        print(outlist2)
        print('--------------------------------------------------------------------------------------------------------------------------------')
        print(labellist2)
        accuracy_per_class2(outlist2, labellist2)

<ipython-input-24-09e4a2d59188>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/39 [00:00<?, ?it/s]

epoch 1 train acc 0.37151616499442586
Label: 중립
Accuracy: 181/495

Label: e
Accuracy: 309/734

Label: s
Accuracy: 280/773

Label: g
Accuracy: 144/476



<ipython-input-24-09e4a2d59188>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/13 [00:00<?, ?it/s]

...

epoch 9 test acc 0.9156167108753315
[tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(1, device='cuda:0'), tensor(0, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(3, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(0, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, d

  0%|          | 0/39 [00:00<?, ?it/s]

epoch 10 train acc 0.9927884615384616
Label: 중립
Accuracy: 489/495

Label: e
Accuracy: 731/734

Label: s
Accuracy: 769/773

Label: g
Accuracy: 471/476



  0%|          | 0/13 [00:00<?, ?it/s]

epoch 10 test acc 0.9177718832891247
[tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(3, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(1, device='cuda:0'), tensor(2, device='cuda:0'), tensor(3, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(3, device='cuda:0'), tensor(0, device='cuda:0'), tensor(3, device='cuda:0'), tensor(3, 

In [ ]:
list1 = [0,0,2,0,0,3,1,1,3,2,3,3,1,1,1,2,2,1,0,3,2,1,0,2,0,1,2]
list(set(list1))
for i in [0,1,2,3]:
  print(i)

0
1
2
3
